# More Kotlin Language features

## If ans swift expressions

`if` and `when` (equivalent of Java's `switch` are expressions)

In [1]:
val total = 5 + 9
val testResult1 = if (total > 3) {
  "Big"
} else {
  "small"
}
testResult1

Big

In [2]:
val greeting = "Bonjour"
val language = when (greeting) {
  "Bonjour", "salut" -> "Français"
  "Good morning" -> "English"
  else -> "Unknown"
}
language

Français

Ternary operation is not available since this can be achieved with a single line `if`

In [3]:
val testResult2 = if (total > 3) "Big" else "small"
testResult2

Big

## Range objects

Ranges are objects that allow to write nice for loops

In [4]:
val range = 1..3
"range: $range, first: ${range.first}, last: ${range.last}, step: ${range.step}"

range: 1..3, first: 1, last: 3, step: 1

For loop are much easier to write and read

In [5]:
for (i in 1..3) println(i)

1
2
3


## Collections

Kotlin, thanks to its standard library, provides out of the box support for collection operations like `map`, `filter`, `reduce`, etc.
The declarative API are somewhat similar Java Streams but can feel more natural to use.

In [6]:
val fantasyNames = listOf(
  "Clebbrer",
  "Sninjur",
  "Moddnaac",
  "Leednat",
  "Gierwyst",
  "Zevurig",
  "Claamparairt",
  "Gufapraam",
  "Riemaprast",
  "Bilanjom"
)

// find names that start with "G" or end with "t" and return them as capital letters
fantasyNames.filter { it.startsWith('G') || it.endsWith('t') }.map { it.uppercase() }


[LEEDNAT, GIERWYST, CLAAMPARAIRT, GUFAPRAAM, RIEMAPRAST]

Ccount the number of vowels. `flapmap` converts a 2d array to a 1d array by concatenating the elements (called a flat operation)

In [7]:
fantasyNames.flatMap { it.toList() }.filter { "aiueo".contains(it) }.map { 1 }.sum()

31

Alternative way using `reduce` instead of `sum`

In [8]:
fantasyNames.flatMap { it.toList() }
  .map { if ("aiueo".contains(it)) 1 else 0 }
  .reduce { acc, current -> acc + current }

31

## Null safety

Kotlin guarantees null safety by distinguishing between nullable and non-nullable types.

In [9]:
var s = "Hello"
// s = null
// at Cell In[29], line 2, column 5: Null can not be a value of a non-null type String

var s2: String? = "Hello"
s2 = null
s2

null

A non-nullable type can be assigned to its nullable equivalent, but not the opposite.

In [10]:
s2 = s
s2

// s = s2
// Type mismatch: inferred type is String? but String was expected

Hello

This means that some functions designed to take non-nullables cannot accept nullables.


In [11]:
//String.format(s2)
//Type mismatch: inferred type is String? but String was expected

In Kotlin, we try to get rid of nullables whenever possible and there are many ways to do it.
Let's see some examples through a function that takes a nullable and returns a non-nullable.

First, let's use smart casting which automatically infers the type based on previous conditions.

In [12]:
fun capitalizeFirstOrDefault(text: String?, default: String): String {
  if (text == null) {
    return default
  }
  // In this point forward text is of type String
  return text.first().uppercase()
}
println(capitalizeFirstOrDefault("Hello", "W"))
println(capitalizeFirstOrDefault(null, "W"))

H
W


We can also use null chaining with the ? operator. The result is still a nullable though. Thus, we need to convert the result into a non-nullable.

In [13]:
fun capitalizeFirstOrDefaultOptChaining(text: String?, default: String): String {
  val firstLetter = text?.first()?.uppercase()
  return if (firstLetter == null) default else firstLetter
}
println(capitalizeFirstOrDefaultOptChaining("Hello", "W"))
println(capitalizeFirstOrDefaultOptChaining(null, "W"))

H
W


The elvis operator `?:` allows to more easily provide a default value to a nullable.

In [14]:
fun capitalizeFirstOrDefaultElvis(text: String?, default: String): String {
  return text?.first()?.uppercase() ?: default
}
println(capitalizeFirstOrDefaultElvis("Hello", "W"))
println(capitalizeFirstOrDefaultElvis(null, "W"))

H
W


We can even make it a single-line function

In [15]:
fun capitalizeFirstSingleLine(text: String?, default: String) = text?.first()?.uppercase() ?: default

println(capitalizeFirstSingleLine("Hello", "W"))
println(capitalizeFirstSingleLine(null, "W"))

H
W


## Object Oriented Programming

- Classes can fit in one line
- `val` generates read-only properties
- `var` generates mutable properties
- getters and setters are generated automatically

In [16]:
class SpaceShip(val maxSpeed: Int, var currentSpeed: Int = 0)

val ship1 = SpaceShip(1000, 0)
println(ship1.maxSpeed)

1000


- `open`: allows inheritance, because classes and properties are final by default
- `init` is the constructor

In [17]:
open class Planet(open val radius: Long) {
  var rotationSpeed = 0

  // Read-only computed property
  val radiusInMeter: Long
    get() = radius * 1000
}

class InhabitedPlanet(override val radius: Long, var population: Long) : Planet(radius) {
  init {
    println("constructor called")
  }

  // We can customize the getter and the setter of a propoerty
  var populationInBillion: Int
    get() = (population / 1_000_000_000).toInt()
    set(value) {
      population = value.toLong() * 1_000_000_000
    }

  fun increasePopulation(amount: Long) {
    population += amount
  }
}

val earth = InhabitedPlanet(6_371, 7_753_000_000_000)
println(earth)
println(earth.radius)
println(earth.population)
earth.population += 1
println(earth.population)
println(earth.populationInBillion)

earth.increasePopulation(100)
println(earth.population)
println(earth.populationInBillion)

constructor called
Line_19_jupyter$InhabitedPlanet@570d827b
6371
7753000000000
7753000000001
7753
7753000000101
7753


Extensions add methods and properties to existing classes. Even ones that we didn't define.

In [18]:
fun String.countLetters(letters: Array<Char>): Int = this.filter { letters.contains(it) }.map { 1 }.sum()

val InhabitedPlanet.hasInhabitants: Boolean
  get() = population > 0

println("Hello".countLetters(arrayOf('H', 'e')))
println(earth.hasInhabitants)

2
true


`data class` are final classes that generate many useful boilerplate code (`hashCode`, `equals`, `copy`, `toString` and `componentN`)

In [19]:
data class SpaceShip(val maxSpeed: Int, var currentSpeed: Int = 0)

val ship1 = SpaceShip(1000, 0)
val ship2 = SpaceShip(1000, 0)
val ship3 = SpaceShip(1000, 10)
println(ship1)
println(ship1 == ship2)
println(ship1 == ship3)

println(ship1.component1())
println(ship1.component2())

// components enable destructuring
val (maxSpeed, currentSpeed) = ship2
println("Max speed: $maxSpeed. Current speed: $currentSpeed")

SpaceShip(maxSpeed=1000, currentSpeed=0)
true
false
1000
0
Max speed: 1000. Current speed: 0
